# Roi analysis

15.12.2022 Sören Doose

## Initial setup

In [ ]:
import sys
from pathlib import Path
import re
import pickle
import logging

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from tqdm.notebook import tqdm
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import locan as lc

In [ ]:
lc.show_versions(dependencies=False)

In [ ]:
# logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger()

## File list

The dataset should have been corrected for drift and chromatic abberation.

Set the directory in which a set of localization files is found.

In [ ]:
directory = Path('.') / '../data'
assert directory.exists()

In [ ]:
files = list(directory.glob('**/*.yaml'))
print(f'Number of files: {len(files)}')
for i, file in enumerate(files):
    print(i,":", file);

## Set up the pipeline

In [ ]:
def computation(self, file):
    """
    Analysis procedure on a LocData object specified by file.
    
    Parameters
    ----------
    self : Pipeline
        Pipeline object that collects results of the analysis procedure.
    file : str
        File path for roi-file.
        
    Returns
    -------
    Pipeline
        The Pipeline object specified by `self`.
    """
    # Prologue
    self.file_indicator = Path(file) #.stem
    
    # Prepare metadata
    metadata_files = list(directory.glob('**/*.toml'))
    metadata_file = lc.find_pattern_upstream(sub_directory=file, pattern="metadata.toml", directory="../data")
    metadata = lc.load_metadata_from_toml(metadata_file)['metadata']
    
    # Load locdata
    logger.debug(f'Loading')
    roi = lc.Roi.from_yaml(path=file)
    roi.reference.file.path = str(Path(file)
                              .relative_to(Path('.'))
                              .with_name(Path(roi.reference.file.path).name)
                             )
    locdata = roi.locdata()
    locdata.meta.MergeFrom(metadata)
    
    # Prerequisites
    if not len(locdata)>1:
        logger.info(f'Not enough data in file: {file}')
        return None
    
    # Select
    condition = '0 < frame < 15_000 and ' \
                '8000 < intensity'
    locdata = lc.select_by_condition(locdata, condition=condition)
    locdata.reduce()
    
    # Update convex hull and alpha shapes
    logger.debug(f'convex_hull')
    locdata.convex_hull
    
    locdata.properties['frame_max'] = locdata.data.frame.max()
    locdata.properties['intensity_min'] = locdata.data.intensity.min()
    locdata.properties['intensity_mean'] = locdata.data.intensity.mean()
    locdata.properties['local_background'] = locdata.data.local_background.mean()
    
    logger.debug(f'LocalizationPropertyCorrelations')
    lpcorr = lc.LocalizationPropertyCorrelations(loc_properties=['intensity', 'local_background']).compute(locdata)
    locdata.properties['correlation_intensity_local_background'] = lpcorr.results.intensity.loc['local_background']
    
    logger.debug(f'LocalizationsPerFrame')
    lpf = lc.LocalizationsPerFrame(norm=locdata.region.region_measure).compute(locdata)
    locdata.properties['loc_density_per_frame_min'] = lpf.results.time_series.min()
    locdata.properties['loc_density_per_frame_max'] = lpf.results.time_series.max()
    locdata.properties['loc_density_per_frame_mean'] = lpf.results.time_series.mean()

    self.locdata_properties = locdata.properties
    
    logger.debug(f'Computation completed for: {file}')
    
    return self

In [ ]:
def save_pickled_pipe(counter: int, pipe: lc.Pipeline, pickle_directory) -> None:
    file_path = Path(pickle_directory) / f'Roi_pipelines_{counter:03}.pickle'
    # inject computation to be pickleable
    pipe.computation = computation
    pipe.parameter['computation'] = computation
    with open(file_path, 'wb') as file:
        pickle.dump(pipe, file, pickle.HIGHEST_PROTOCOL)

## Run pipeline

In [ ]:
logger.setLevel(logging.INFO)  # alternative logging.WARNING

In [ ]:
%%time
def worker(i, file):
    logging.basicConfig(level=logging.INFO)
    logger.info(f'Processing {i} : {file}')
    try:
        pipe = lc.Pipeline(computation=computation, file=file).compute()
        # save_pickled_pipe(counter=i, pipe=pipe, pickle_directory=pickle_directory)
        return pipe
    except Exception as e:
        raise(e)
        logger.warning(f'Error in {file}')
        return None

pipes = [worker(i, file) for i, file in tqdm(enumerate(files[0:1]), desc='Processed files:')]
print(f'Number of pipes: {len(pipes)}')

### Multiprocessing with ray

### Remove None from pipes

In [ ]:
print(f'Number of pipes: {len(pipes)}')
pipes = [pipe for pipe in pipes if pipe]
print(f'Number of pipes that are not None: {len(pipes)}')

### Pipeline attributes

In [ ]:
[attr for attr in dir(pipes[0]) if not attr.startswith('__') and not attr.endswith('__')]

## Save pickled pipes

Save pickled pipes if not done during computation pipe-by-pipe.

## Load pickled pipes

## Data presentation

### Extract from pipes

In [ ]:
def collect_scalars(pipelines):
    """
    Collect scalar properties from Pipeline objects and assemble them in a pandas.DataFrame.
    
    Parameters
    ----------
    pipelines : list(Pipeline)
        Pipeline objects.
        
    Returns
    -------
    pandas.DataFrame
    """
    dictionaries = []
    for pipe in tqdm(pipelines, desc='Processed pipelines:'):

        new_dict = {
            'files': pipe.file_indicator
            }
        new_dict.update(pipe.locdata_properties)

        dictionaries.append(new_dict)
        
    return pd.DataFrame(dictionaries)

In [ ]:
scalars_df = collect_scalars(pipes)

In [ ]:
scalars_df

### Compare for files

Reduce the file name to group identifier:

In [ ]:
#choices = ['*/' + str(file.parent.relative_to('../data')).replace('\\', '/') + '/*' for file in files]
choices = ['*/' + str(file.relative_to('../data')).replace('\\', '/') for file in files]

choices_name = [choice[2:-2] for choice in choices]

choices_name = ['CD18' if ('CD18_' in cn) else cn for cn in choices_name]
    
conditions = [[f.match(name) for f in scalars_df['files']] for name in choices]
scalars_df['sample'] = np.select(conditions, choices_name, default=None)

In [ ]:
grouped = scalars_df.groupby('sample')

In [ ]:
print('Number of groups:', len(grouped.groups))
list(grouped.groups)

### Descriptive statistics

#### Mean values:

In [ ]:
grouped.size()

In [ ]:
grouped.mean()

In [ ]:
grouped.median()

#### Standard errors of the mean:

In [ ]:
grouped.sem()

### Statistical plots

In [ ]:
scalars_df.columns

In [ ]:
blacklist = [
    'files',
    'position_x',
    'position_y',
    'region_measure_bb',
    'localization_density_bb',
    'subregion_measure_bb',
    'localization_density_ch',
    'sample'
]
columns_of_interest = [column for column in scalars_df.columns if column not in blacklist]
columns_of_interest

In [ ]:
import plotly.express as px

n_elements = len(columns_of_interest)
n_cols = 1
n_rows = -(-n_elements // n_cols)

fig = make_subplots(rows=n_rows, cols=n_cols, subplot_titles=columns_of_interest, vertical_spacing=0.01)

colors = px.colors.qualitative.T10 * 5
assert len(colors) >= len(grouped)

for column, indices in zip(columns_of_interest, lc.iterate_2d_array(n_elements, n_cols)):
    for (key, value), color in zip(grouped[column], colors):
        fig.add_trace(
            go.Box(x=value, name=key, boxpoints='all', marker_size=5, width=0.5, marker_color =color),
            row=indices[0]+1, col=indices[1]+1
            )
        fig.update_layout(boxmode='group', showlegend=False)

fig.update_layout(height=10_000, width=1000, title_text="Averaged properties for each experimental group")
go.FigureWidget(fig).show()